<a href="https://colab.research.google.com/github/saatweek/Intel-Image-Classification/blob/main/Accuracy_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#The dataset is stored in my Google Drive, so we'll first have to connect this notebook 
#to the Google drive to access the files in the drive from this notebook 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#importing dependencies

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import plotly.graph_objects as go
import tensorflow_hub as hub
import plotly.graph_objects as go

In [3]:
train_dir = '/content/drive/My Drive/Colab Notebooks/Personal Projects/Intel Image Classification/seg_train'
validation_dir = '/content/drive/My Drive/Colab Notebooks/Personal Projects/Intel Image Classification/seg_test'

train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=0.3, width_shift_range=0.3,
    height_shift_range=0.3,shear_range=0.3, zoom_range=0.3, horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=0.3, width_shift_range=0.3,
    height_shift_range=0.3,shear_range=0.3, zoom_range=0.3, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150, 150), batch_size = 128, class_mode='sparse')

validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size = (150, 150), batch_size = 128, class_mode='sparse')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [4]:
#Inception Network

url = 'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4'

base_model = hub.KerasLayer(url, input_shape = (150, 150, 3))

inception_model = tf.keras.Sequential([base_model, 
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(6, activation='softmax')])

inception_model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics = 'acc')
inception_history = inception_model.fit(train_generator, steps_per_epoch=20, epochs=10, validation_data = validation_generator)

Epoch 1/10
20/20 [==============================] - 1318s 66s/step - loss: 0.9916 - acc: 0.6270 - val_loss: 0.5559 - val_acc: 0.8063
Epoch 2/10
20/20 [==============================] - 485s 24s/step - loss: 0.5135 - acc: 0.8109 - val_loss: 0.4945 - val_acc: 0.8143
Epoch 3/10
20/20 [==============================] - 377s 19s/step - loss: 0.5098 - acc: 0.8207 - val_loss: 0.4667 - val_acc: 0.8273
Epoch 4/10
20/20 [==============================] - 312s 16s/step - loss: 0.4557 - acc: 0.8359 - val_loss: 0.4505 - val_acc: 0.8380
Epoch 5/10
20/20 [==============================] - 253s 13s/step - loss: 0.4315 - acc: 0.8492 - val_loss: 0.4685 - val_acc: 0.8297
Epoch 6/10
20/20 [==============================] - 214s 11s/step - loss: 0.4320 - acc: 0.8469 - val_loss: 0.4550 - val_acc: 0.8400
Epoch 7/10
20/20 [==============================] - 173s 9s/step - loss: 0.4129 - acc: 0.8477 - val_loss: 0.4323 - val_acc: 0.8410
Epoch 8/10
20/20 [==============================] - 141s 7s/step - loss: 0.4

In [5]:
#Convolution Network

conv_model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(256, activation='relu'),
                                    tf.keras.layers.Dense(6, activation = 'softmax')])

conv_model.compile(loss = 'sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics = 'acc')

conv_history = conv_model.fit(train_generator, steps_per_epoch=20, epochs=10, validation_data = validation_generator)

Epoch 1/10
20/20 [==============================] - 107s 5s/step - loss: 1.5789 - acc: 0.3391 - val_loss: 1.5554 - val_acc: 0.4137
Epoch 2/10
20/20 [==============================] - 85s 4s/step - loss: 1.2839 - acc: 0.4770 - val_loss: 1.2558 - val_acc: 0.5043
Epoch 3/10
20/20 [==============================] - 82s 4s/step - loss: 1.1626 - acc: 0.5437 - val_loss: 1.2054 - val_acc: 0.5343
Epoch 4/10
20/20 [==============================] - 65s 3s/step - loss: 1.1170 - acc: 0.5601 - val_loss: 1.1163 - val_acc: 0.5577
Epoch 5/10
20/20 [==============================] - 61s 3s/step - loss: 1.0894 - acc: 0.5760 - val_loss: 1.1055 - val_acc: 0.5653
Epoch 6/10
20/20 [==============================] - 56s 3s/step - loss: 1.0367 - acc: 0.6006 - val_loss: 1.0619 - val_acc: 0.5970
Epoch 7/10
20/20 [==============================] - 54s 3s/step - loss: 1.0001 - acc: 0.6180 - val_loss: 0.9548 - val_acc: 0.6257
Epoch 8/10
20/20 [==============================] - 48s 2s/step - loss: 0.9521 - acc: 0.6

In [8]:
#Plotting the comparison Graph

fig = go.Figure()
x = np.linspace(1,100, 100)
y1 = conv_history.history['acc']
y2 = inception_history.history['acc']
fig.add_trace(go.Scatter(x = x, y = y1, name = 'Convolution accuracy')),
fig.add_trace(go.Scatter(x = x, y = y2, name = 'Inception Network accuracy')),
fig.update_layout(xaxis_title = 'Epochs', yaxis_title = 'accuracy', title = 'Accuracy of Model on Training Data')
fig.show()

In [10]:
fig = go.Figure()
x = np.linspace(1,100, 100)
y1 = conv_history.history['val_acc']
y2 = inception_history.history['val_acc']
fig.add_trace(go.Scatter(x = x, y = y1, name = 'Convolution accuracy')),
fig.add_trace(go.Scatter(x = x, y = y2, name = 'Inception Network accuracy')),
fig.update_layout(xaxis_title = 'Epochs', yaxis_title = 'accuracy', title = 'Accuracy of Model on Validation Data')
fig.show()